In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoVanillaTransformer
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.core import NeuralForecast

from datetime import datetime, timedelta
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from neuralforecast.losses.pytorch import MQLoss
from tqdm import tqdm

In [2]:
# Load and process the MEASLES_ARIZONA data
df = pd.read_csv("outbreaks_disease_location.csv")
value_columns = [str(i) for i in range(60)]
series_values = df[value_columns].fillna(0).astype(float)
start_dates = pd.to_datetime(df["start_date"])

# Shuffle and split
shuffled_indices = df.sample(frac=1, random_state=42).index
split_point = int(0.8 * len(df))
train_indices = shuffled_indices[:split_point]
test_indices = shuffled_indices[split_point:]

In [3]:
train_records = []
for i, row in series_values.iloc[train_indices].iterrows():
    dates = pd.date_range(start="2000-01-01", periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        train_records.append({"unique_id": f"Y{i+1}", "ds": dates[t], "y": value})
df_train = pd.DataFrame(train_records)

test_start_dates = start_dates.loc[test_indices] - pd.Timedelta(weeks=4)
df_test_all = []

for idx in test_indices:
    start_date = test_start_dates.loc[idx]
    row = series_values.loc[idx]
    dates = pd.date_range(start=start_date, periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        df_test_all.append({"unique_id": f"Y_{idx}", "ds": dates[t], "y": value})

df_test_all = pd.DataFrame(df_test_all)

In [4]:
df_train["date"] = pd.to_datetime(df_train["ds"])
df_train.set_index("date", inplace = True)

df_test_all["date"] = pd.to_datetime(df_test_all["ds"])
df_test_all.set_index("date", inplace = True)

In [5]:
df_test_all

,unique_id,ds,y
date,,,
2022-08-13,Y_5607,2022-08-13,20775.0
2022-08-20,Y_5607,2022-08-20,16570.0
2022-08-27,Y_5607,2022-08-27,13995.0
2022-09-03,Y_5607,2022-09-03,12145.0
2022-09-10,Y_5607,2022-09-10,10686.0
...,...,...,...
1939-05-20,Y_7270,1939-05-20,0.0
1939-05-27,Y_7270,1939-05-27,0.0
1939-06-03,Y_7270,1939-06-03,0.0


In [6]:
class FixedModelVanillaTransformerProcessor:
    def __init__(self, dates=[]):
        self.dates  = dates
        self.forecasts, self.eval_pairs = [], []
        self.testset = None
        self.nf, self.model = None, None
        self.selected_input_size = None
        self.maes, self.mses, self.mapes, self.nmses = [], [], [], []
        self.metrics_df  = pd.DataFrame(columns=["Unique_id","Reference Date","MAE","MSE","MAPE","NMSE"])
        self.display_df  = pd.DataFrame(columns=["Unique_id","Reference Date","Target End Date","GT","Quantile","Prediction"])
        self.wis_df      = pd.DataFrame()

    # ──────────────────────────────────────────
    def create_fixed_model(self, df_train, df_test, h: int, freq: str, level=[]):
        min_len = df_train.groupby("unique_id").size().min()
        max_in  = max(8, min_len - h - 1)
        self.testset = df_test

        def cfg(t):
            return {
                "input_size"   : t.suggest_int("input_size", 8, max_in),
                "learning_rate": t.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
                "batch_size"   : t.suggest_categorical("batch_size",[16,32,64]),
                "random_seed"  : t.suggest_int("random_seed",1,99999),
                "accelerator":"gpu","devices":1,"strategy":"auto"
            }

        model = AutoVanillaTransformer(
                    h=h, backend="optuna",
                    loss=MQLoss(level=level) if level else None,
                    config=cfg)

        self.nf = NeuralForecast(models=[model], freq=freq)
        self.nf.fit(df=df_train)
        self.model = self.nf.models[0].model
        self.selected_input_size = self.model.hparams["input_size"]

    # ──────────────────────────────────────────
    def sliding_batch_by_index(self, h: int = 4):
        uid_dates = {u: g["ds"].sort_values().to_numpy()
                     for u, g in self.testset.groupby("unique_id")}
        T, s = min(len(d) for d in uid_dates.values()), self.selected_input_size
        rows = []

        for t in tqdm(range(s, T - h + 1), desc="index t"):
            batch, refs = [], []
            for uid, dates in uid_dates.items():
                win, ref = dates[t-s:t], dates[t-1]
                df_u = self.testset[self.testset["unique_id"] == uid]
                df_in = df_u[df_u["ds"].isin(win)].copy()
                if len(df_in) == s:
                    df_in["unique_id"] = uid
                    batch.append(df_in); refs.append((uid, ref))
            if not batch: continue

            fcst = self.nf.predict(df=pd.concat(batch)).set_index("ds")
            for uid, ref in refs:
                f  = fcst[fcst["unique_id"] == uid].reset_index()
                gt = (self.testset[self.testset["unique_id"] == uid]
                      .set_index("ds").loc[f["ds"]].reset_index())
                merged = pd.merge(f, gt, on=["unique_id","ds"], how="inner")
                merged["Reference Date"] = pd.to_datetime(ref).strftime("%Y-%m-%d")
                rows.append(merged)

        return pd.concat(rows, ignore_index=True)

    # ──────────────────────────────────────────
    def create_display_df(self, merged_df, model_name="AutoVanillaTransformer"):
        recs = []
        for _, g in tqdm(merged_df.groupby("unique_id"), desc="display"):
            uid = g["unique_id"].iloc[0]; ref = g["Reference Date"].iloc[0]
            for col in g.columns:
                if not col.startswith(model_name): continue
                if col.endswith("median") or col == model_name:
                    q = 0.5
                elif "-lo-" in col:
                    lvl = int(col.split("-")[-1]); q = (1-lvl/100)/2
                elif "-hi-" in col:
                    lvl = int(col.split("-")[-1]); q = 1-(1-lvl/100)/2
                else:
                    continue
                tmp = g[["ds", col, "y"]].rename(columns={"ds":"Target End Date",
                                                           col:"Prediction",
                                                           "y":"GT"})
                tmp["Unique_id"] = uid
                tmp["Reference Date"] = ref
                tmp["Quantile"] = q
                recs.append(tmp)
        self.display_df = (pd.concat(recs)
                           .loc[:,["Unique_id","Reference Date","Target End Date",
                                   "GT","Quantile","Prediction"]]
                           .sort_values(["Unique_id","Reference Date",
                                         "Target End Date","Quantile"])
                           .reset_index(drop=True))

    # ──────────────────────────────────────────
    def efficient_compute_wis(self):
        if self.display_df.empty:
            raise ValueError("Run create_display_df first.")
        rows, df = [], self.display_df.sort_values(
            ["Unique_id","Reference Date","Target End Date","Quantile"])
        for (uid, ref, tgt), grp in tqdm(df.groupby(
                ["Unique_id","Reference Date","Target End Date"]), desc="WIS"):
            gt = grp["GT"].iloc[0]
            preds = grp.set_index("Quantile")["Prediction"]
            if 0.5 not in preds.index: continue
            rows.append({"Unique_id":uid,"Reference Date":ref,
                         "Target End Date":tgt,"GT":gt,
                         "WIS": abs(preds[0.5]-gt)})   # AE shortcut
        self.wis_df = pd.DataFrame(rows)


In [7]:
test_ids = [f"Y_{i}" for i in test_indices]

In [8]:
processor = FixedModelVanillaTransformerProcessor()

In [9]:
processor.create_fixed_model(df_train, df_test_all, h=4, freq="W-SAT", level = [10,20,30,40,50,60,70,80, 85,90,95])

[I 2025-06-25 18:27:33,348] A new study created in memory with name: no-name-0f945690-9d53-4675-86ed-0809d3216336
[rank: 0] Seed set to 72359
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | loss          | MQLoss        | 23     | train
1 | padder_train  | ConstantPad1d | 0      | train
2 | scaler        | TemporalNorm  | 0      | train
3 | enc_embedding | DataEmbedding | 384    | train
4 | dec_embedding | DataEmbedding | 384    | train
5 | encoder       | TransEncoder  | 150 K  | train
6 | decoder       | TransDecoder  | 144 K  | train
--------------------------------------------------------
295 K     Trainable params
23        Non-trainable params
295 K     Total params
1.181     Total estimated model params size (MB)
67        Modules in train mode
0         Mod

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=5000` reached.


In [10]:
results = []

In [11]:
processor.selected_input_size

37

In [12]:
all_results = processor.sliding_batch_by_index(h=4)

index t:   0%|          | 0/20 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:   5%|▌         | 1/20 [00:38<12:08, 38.33s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:  10%|█         | 2/20 [01:13<10:55, 36.41s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

index t:  90%|█████████ | 18/20 [10:31<01:10, 35.30s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:  95%|█████████▌| 19/20 [11:06<00:35, 35.15s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t: 100%|██████████| 20/20 [11:40<00:00, 35.02s/it]


In [13]:
all_results.to_csv('transformer_concatenated.csv')

In [14]:
all_results

,ds,unique_id,AutoVanillaTransformer-median,AutoVanillaTransformer-lo-95,AutoVanillaTransformer-lo-90,AutoVanillaTransformer-lo-85,AutoVanillaTransformer-lo-80,AutoVanillaTransformer-lo-70,AutoVanillaTransformer-lo-60,AutoVanillaTransformer-lo-50,...,AutoVanillaTransformer-hi-40,AutoVanillaTransformer-hi-50,AutoVanillaTransformer-hi-60,AutoVanillaTransformer-hi-70,AutoVanillaTransformer-hi-80,AutoVanillaTransformer-hi-85,AutoVanillaTransformer-hi-90,AutoVanillaTransformer-hi-95,y,Reference Date
0,2011-06-04,Y_1,-0.024053,-0.005522,-0.003924,-0.007085,-0.000994,-0.001545,-0.015939,-0.025495,...,-0.003088,0.004958,0.000061,0.006375,0.026990,0.081239,0.148164,0.325162,0.065660,2011-05-28
1,2011-06-11,Y_1,-0.028105,-0.005488,-0.003948,-0.007103,-0.001045,-0.001747,-0.017021,-0.027006,...,-0.007020,-0.000611,-0.006464,-0.000342,0.017591,0.069436,0.136991,0.310100,0.123609,2011-05-28
2,2011-06-18,Y_1,-0.029349,-0.005497,-0.003979,-0.007091,-0.001002,-0.001793,-0.017281,-0.027317,...,-0.007819,-0.001425,-0.007514,-0.000900,0.015647,0.068137,0.135728,0.310132,0.051706,2011-05-28
3,2011-06-25,Y_1,-0.049488,-0.006386,-0.004485,-0.007100,-0.001178,-0.002059,-0.025117,-0.041203,...,-0.022886,-0.009572,-0.009491,0.009618,0.026001,0.089421,0.179904,0.368561,0.148478,2011-05-28
4,2022-01-29,Y_1000,-0.139586,-0.010239,-0.009086,-0.008495,-0.003334,-0.004922,-0.057802,-0.097218,...,-0.086096,-0.024723,-0.085364,-0.083262,-0.094954,0.025514,0.042620,0.149454,0.000000,2022-01-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172795,1955-04-16,Y_9998,0.076860,-0.005600,-0.002986,-0.006967,-0.000625,-0.001371,-0.018775,-0.023174,...,0.217644,0.293083,0.361197,0.478012,0.616369,0.771644,0.917993,1.247223,0.000000,1955-03-19
172796,1957-03-02,Y_9999,0.696084,-0.004563,-0.001413,-0.005970,0.000615,0.004080,0.106021,0.215700,...,1.137400,1.297039,1.448622,1.664441,1.906354,2.136541,2.312841,2.737881,0.000000,1957-02-23
172797,1957-03-09,Y_9999,0.448622,-0.004343,-0.002438,-0.006710,-0.000462,0.000332,0.054123,0.115192,...,0.789392,0.907910,1.014044,1.184199,1.370610,1.551191,1.707071,2.059272,0.000000,1957-02-23
172798,1957-03-16,Y_9999,0.444779,-0.004347,-0.002454,-0.006691,-0.000414,0.000292,0.053306,0.113840,...,0.784916,0.903458,1.009223,1.180117,1.365795,1.547605,1.704012,2.058139,0.000000,1957-02-23


In [15]:
df = all_results.rename(columns={
    "ds"        : "Target End Date",
    "unique_id" : "Unique_id",
    "y"         : "GT"
})

# long‐format all quantile columns (everything that starts with AutoNBEATS-)
quantile_cols = [c for c in df.columns
                 if c.startswith("AutoVanillaTransformer-")]

df_long = df.melt(
    id_vars = ["Unique_id", "Reference Date", "Target End Date", "GT"],
    value_vars = quantile_cols,
    var_name = "QuantileLabel",
    value_name = "Prediction"
)

def to_quantile(label: str) -> float:
    if "median" in label: return 0.5
    if "-lo-" in label:
        lvl = int(label.split("-")[-1]); return (1 - lvl/100) / 2
    if "-hi-" in label:
        lvl = int(label.split("-")[-1]); return 1 - (1 - lvl/100) / 2
    raise ValueError(label)

df_long["Quantile"] = df_long["QuantileLabel"].apply(to_quantile)

rows = []

group_iter = df_long.groupby(["Unique_id", "Reference Date", "Target End Date"])
for (uid, ref_date, tgt_date), g in tqdm(group_iter,
                                         total=len(group_iter),
                                         desc="point-wise metrics"):

    gt = g["GT"].iloc[0]
    preds = g.set_index("Quantile")["Prediction"]

    if 0.5 not in preds:          # need the median
        continue

    # point errors
    y_pred = preds[0.5]
    mae  = abs(y_pred - gt)
    mse  = (y_pred - gt) ** 2
    mape = abs((y_pred - gt) / gt) if gt else np.nan
    nmse = np.nan                # single point ⇒ var = 0

    # WIS for this timestamp
    qs  = sorted(q for q in preds.index if q != 0.5)
    n   = len(qs) // 2
    ae  = mae
    int_scores = 0
    for i in range(n):
        lo_q, hi_q = qs[i], qs[-(i + 1)]
        lo, hi     = preds[lo_q], preds[hi_q]
        alpha      = hi_q - lo_q
        int_scores += (hi - lo) \
                    + (2/alpha) * max(lo - gt, 0) \
                    + (2/alpha) * max(gt - hi, 0)

    wis = (ae + int_scores) / (1 + n)

    rows.append({
        "Unique_id"      : uid,
        "Reference Date" : ref_date,
        "Target End Date": tgt_date,
        "GT"             : gt,
        "Prediction"     : y_pred,
        "MAE"            : mae,
        "MSE"            : mse,
        "MAPE"           : mape,
        "NMSE"           : nmse,
        "WIS"            : wis
    })

pointwise_df = pd.DataFrame(rows).sort_values(
    ["Unique_id", "Reference Date", "Target End Date"])

point-wise metrics: 100%|██████████| 172800/172800 [01:23<00:00, 2058.66it/s]


In [16]:
pointwise_df

,Unique_id,Reference Date,Target End Date,GT,Prediction,MAE,MSE,MAPE,NMSE,WIS
0,Y_1,2011-05-28,2011-06-04,0.065660,-0.024053,0.089713,0.008048,1.366328,NaN,0.406956
1,Y_1,2011-05-28,2011-06-11,0.123609,-0.028105,0.151714,0.023017,1.227366,NaN,0.700691
2,Y_1,2011-05-28,2011-06-18,0.051706,-0.029349,0.081056,0.006570,1.567619,NaN,0.362373
3,Y_1,2011-05-28,2011-06-25,0.148478,-0.049488,0.197966,0.039191,1.333301,NaN,0.896109
4,Y_1,2011-06-04,2011-06-11,0.123609,-0.045054,0.168663,0.028447,1.364485,NaN,0.794678
...,...,...,...,...,...,...,...,...,...,...
172795,Y_9999,1957-02-16,1957-03-16,0.000000,0.339567,0.339567,0.115306,NaN,NaN,1.717339
172796,Y_9999,1957-02-23,1957-03-02,0.000000,0.696084,0.696084,0.484534,NaN,NaN,3.236789
172797,Y_9999,1957-02-23,1957-03-09,0.000000,0.448622,0.448622,0.201262,NaN,NaN,2.154832
172798,Y_9999,1957-02-23,1957-03-16,0.000000,0.444779,0.444779,0.197828,NaN,NaN,2.141281


In [17]:
pointwise_df.to_csv('Transformer_ALL_STATS.csv')